# H5 Data Analysis and Preparation

Adapted from https://github.com/MLforHealth/MIMIC_Extract/blob/master/notebooks/Baselines%20for%20Mortality%20and%20LOS%20prediction%20-%20GRU-D.ipynb

Create an environment: `/opt/miniconda3/bin/conda create --name=mimic-extract-2`

Install packages:
- Analysis: `conda install pandas pytables scipy numpy ipython ipykernel pytorch imbalanced-learn`

In [11]:
print("hello world")

hello world


In [12]:
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss, torch

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [13]:
DATA_FILEPATH     = '/home/iandre3/mimic-extract-data/all_hourly_data.h5'
GAP_TIME          = 2  # In hours
WINDOW_SIZE       = 48 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '2'

# os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [14]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [15]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

CPU times: user 7.68 s, sys: 5.27 s, total: 13 s
Wall time: 32 s


In [16]:
data_full_lvl2.head()

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   

LEVEL2                                 white blood cell count urine           \
Aggregation Function                                          count mean std   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                                 0.0  NaN NaN   
                              1                                 0.0  NaN NaN   
                              2                                 0.0  NaN NaN   
                              3                                 0.0  NaN NaN   
                              4                                 0.0  NaN NaN   

LEVEL2                                    ph                 ph urine           
Aggregation Function                   count  mean       std    count mean std  
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0          9.0  7.40  0.147733      1.0  5.0 NaN  
                              1          0.0   NaN       NaN      0.0  NaN NaN  
                              2          3.0  7.26  0.000000      0.0  NaN NaN  
                              3          0.0   NaN       NaN      0.0  NaN NaN  
                              4          0.0   NaN       NaN      0.0  NaN NaN  

[5 rows x 312 columns]

In [17]:
statics.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [18]:
%%time 
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    print("1")
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    print("2")
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    print("3")
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    print("4")
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)
    print("5")
    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    print("6")
    df_out.sort_index(axis=1, inplace=True)
    return df_out

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


Data Loaded - Start cleaning

In [19]:
%%time

Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_icu']]
Ys.astype(float)
print(1)
lvl2 = data_full_lvl2[
    (data_full_lvl2.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (data_full_lvl2.index.get_level_values('hours_in') < WINDOW_SIZE)
]
print(2)
train_frac = 1.0
lvl2_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train = int(train_frac * N)
train_subj = subjects[:N_train]

[(lvl2_train, ), (Ys_train, )] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, )] \
    for df in (lvl2, Ys)
]

idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)

lvl2_train.loc[:, idx[:,'mean']] = (lvl2_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

1
2
CPU times: user 1min 38s, sys: 2.42 s, total: 1min 40s
Wall time: 1min 40s


In [20]:
%%time

lvl2_train = simple_imputer(lvl2_train)

assert not lvl2_train.isnull().any().any()

1
2


<timed exec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


3


<timed exec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4
5
6
CPU times: user 10min 21s, sys: 9.88 s, total: 10min 31s
Wall time: 10min 46s


In [21]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_icu']]
Ys.astype(float)
Ys_train = Ys[Ys.index.get_level_values('subject_id').isin(train_subj)]

In [22]:
Ys_train.head()

,,,mort_icu
subject_id,hadm_id,icustay_id,
3,145834,211552,0
6,107064,228232,0
9,150750,220597,1
12,112213,232669,0
13,143045,263738,0


In [23]:
pd.set_option('display.max_columns', None)
lvl2_train.head(n=48)

LEVEL2                                 alanine aminotransferase            \
Aggregation Function                                       mask      mean   
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0                             1.0 -0.283859   
                              1                             0.0 -0.283859   
                              2                             0.0 -0.283859   
                              3                             0.0 -0.283859   
                              4                             0.0 -0.283859   
                              5                             0.0 -0.283859   
                              6                             0.0 -0.283859   
                              7                             0.0 -0.283859   
                              8                             0.0 -0.283859   
                              9                             0.0 -0.283859   
                              10                            0.0 -0.283859   
                              11                            0.0 -0.283859   
                              12                            0.0 -0.283859   
                              13                            0.0 -0.283859   
                              14                            0.0 -0.283859   
                              15                            0.0 -0.283859   
                              16                            0.0 -0.283859   
                              17                            0.0 -0.283859   
                              18                            0.0 -0.283859   
                              19                            0.0 -0.283859   
                              20                            0.0 -0.283859   
                              21                            0.0 -0.283859   
                              22                            0.0 -0.283859   
                              23                            0.0 -0.283859   
                              24                            0.0 -0.283859   
                              25                            0.0 -0.283859   
                              26                            0.0 -0.283859   
                              27                            0.0 -0.283859   
                              28                            0.0 -0.283859   
                              29                            0.0 -0.283859   
                              30                            0.0 -0.283859   
                              31                            0.0 -0.283859   
                              32                            1.0 -0.288781   
                              33                            0.0 -0.288781   
                              34                            0.0 -0.288781   
                              35                            0.0 -0.288781   
                              36                            0.0 -0.288781   
                              37                            0.0 -0.288781   
                              38                            0.0 -0.288781   
                              39                            0.0 -0.288781   
                              40                            0.0 -0.288781   
                              41                            0.0 -0.288781   
                              42                            0.0 -0.288781   
                              43                            0.0 -0.288781   
                              44                            0.0 -0.288781   
                              45                            0.0 -0.288781   
                              46                            0.0 -0.288781   
                              47                            0.0 -0.288781   

LEVEL2                                                     albumin      

In [28]:
for name in [("Ys_train", Ys_train), ("lvl2_train", lvl2_train)]:
    with open(f'data/{name[0]}.pkl', 'wb+') as f:
        pickle.dump(name[1], f)

## Cleanup

In [29]:
data = lvl2_train.sort_values(['subject_id', 'hadm_id', 'icustay_id']).reset_index(drop=True).droplevel(level="LEVEL2", axis=1)
labels = Ys_train.sort_values(['subject_id', 'hadm_id', 'icustay_id'])

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [31]:
def print_data_info(features, masks, labels):
    print(f"Shape of features: {features.size()}")
    print(f"Shape of masks: {masks.size()}")
    print(f"Shape of labels: {labels.size()}")
    
    print(f"Labels (format: [(label, count)]: {list(zip(*torch.unique(labels, return_counts = True)))}")

    assert len(features) == len(labels)
    assert len(masks) == len(labels)
    assert features.size() == masks.size()

In [32]:
features = data["mean"].to_numpy()
masks = data["mask"].to_numpy()

features = np.split(features, [48 * i for i in range(1, len(features) // 48)])
masks = np.split(masks, [48 * i for i in range(1, len(masks) // 48)])

features = torch.tensor(features, dtype=torch.float).to(device)
masks = torch.tensor(masks, dtype=torch.float).to(device)
labels = torch.squeeze(torch.tensor(labels.to_numpy(), dtype=torch.float).to(device))

print_data_info(features, masks, labels)

/tmp/ipykernel_3662323/2999371896.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1648016058912/work/torch/csrc/utils/tensor_new.cpp:201.)
  features = torch.tensor(features, dtype=torch.float).to(device)


Shape of features: torch.Size([15591, 48, 104])
Shape of masks: torch.Size([15591, 48, 104])
Shape of labels: torch.Size([15591])
Labels (format: [(label, count)]: [(tensor(0.), tensor(14241)), (tensor(1.), tensor(1350))]


In [33]:
train_features, test_features, train_masks, test_masks, train_labels, test_labels = train_test_split(
    features, masks, labels, test_size=0.1, random_state=42)
train_features, val_features, train_masks, val_masks, train_labels, val_labels = train_test_split(
    train_features, train_masks, train_labels, test_size=0.1, random_state=42)

print("Training dataset")
print_data_info(train_features, train_masks, train_labels)
print("\nTest dataset")
print_data_info(test_features, test_masks, test_labels)
print("\nValidation dataset")
print_data_info(val_features, val_masks, val_labels)

Training dataset
Shape of features: torch.Size([12627, 48, 104])
Shape of masks: torch.Size([12627, 48, 104])
Shape of labels: torch.Size([12627])
Labels (format: [(label, count)]: [(tensor(0.), tensor(11531)), (tensor(1.), tensor(1096))]

Test dataset
Shape of features: torch.Size([1560, 48, 104])
Shape of masks: torch.Size([1560, 48, 104])
Shape of labels: torch.Size([1560])
Labels (format: [(label, count)]: [(tensor(0.), tensor(1431)), (tensor(1.), tensor(129))]

Validation dataset
Shape of features: torch.Size([1404, 48, 104])
Shape of masks: torch.Size([1404, 48, 104])
Shape of labels: torch.Size([1404])
Labels (format: [(label, count)]: [(tensor(0.), tensor(1279)), (tensor(1.), tensor(125))]


In [34]:
def oversample_data(features, masks, labels):
    X = list(zip(features, masks))
    ros = RandomOverSampler(random_state=0)
    X_resampled, labels_resampled = ros.fit_resample(X, labels)
    features_resampled, masks_resampled = list(zip(*X_resampled))

    features_resampled = np.array([np.array(xi) for xi in features_resampled])
    masks_resampled = np.array([np.array(xi) for xi in masks_resampled])

    features = torch.tensor(features_resampled, dtype=torch.float).to(device)
    masks = torch.tensor(masks_resampled, dtype=torch.float).to(device)
    labels = torch.tensor(labels_resampled, dtype=torch.float).to(device)

    return features, masks, labels

train_features, train_masks, train_labels = oversample_data(train_features, train_masks, train_labels)
print_data_info(train_features, train_masks, train_labels)

/home/iandre3/.conda/envs/mimic-extract-2/lib/python3.9/site-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)
/home/iandre3/.conda/envs/mimic-extract-2/lib/python3.9/site-packages/sklearn/utils/validation.py:746: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray

Shape of features: torch.Size([23062, 48, 104])
Shape of masks: torch.Size([23062, 48, 104])
Shape of labels: torch.Size([23062])
Labels (format: [(label, count)]: [(tensor(0.), tensor(11531)), (tensor(1.), tensor(11531))]


In [35]:
for name in [
    ("train_features", train_features),
    ("train_masks", train_masks),
    ("train_labels", train_labels),
    ("test_features", test_features),
    ("test_masks", test_masks),
    ("test_labels", test_labels),
    ("val_features", val_features),
    ("val_masks", val_masks),
    ("val_labels", val_labels)
]:
    with open(f'data/{name[0]}.pkl', 'wb') as f:
        pickle.dump(name[1], f)